# PeDistance Linear Regression

### Initial settings

In [39]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.tensor(df.to_numpy()).float().to(device)

### Import Dataset

In [32]:
fields = [
    'Distance', 
    'PedTopLeftX', 
    'PedTopLeftY', 
    'PedTopRightX', 
    'PedTopRightY',
    'PedBottomLeftX', 
    'PedBottomLeftY',
    'PedBottomRightX',
    'PedBottomRightY'
]

df = pd.read_csv (r'./Dataset.csv', usecols=fields)
print(df.shape)

(689, 9)


### Splitting dataset


In [33]:
train, test = train_test_split(df, test_size = 0.3)

In [35]:
Y_training = df_to_tensor(train.Distance)
Y_testing = df_to_tensor(test.Distance)

print(Y_training.shape)
print(Y_testing.shape)

torch.Size([482])
torch.Size([207])


#### Getting width  height from raw 2D screen points

In [36]:
X_training = df_to_tensor(train.apply(lambda el :  pd.Series([abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], index=['width', 'height']), axis=1))
X_testing = df_to_tensor(test.apply(lambda el :  pd.Series([abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], index=['width', 'height']), axis=1))

print(X_training.shape)
print(X_testing.shape)

torch.Size([482, 2])
torch.Size([207, 2])


### Linear Regression

In [37]:
class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(in_size,out_size)
    
    def forward(self,x):
        result = self.linear(x)
        return result
    
class AverageValueMeter():
    def __init__(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, value, num):
        self.sum += value*num
        self.num += num

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None

In [38]:
lr = 0.0001
momentum = 0.5
epochs = 200000

writer = SummaryWriter('logs/linear_regressor_lr' + str(lr) + '_momentum' + str(momentum) + '_epochs' + str(epochs))

#normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)

X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds

reg = LinearRegressor(2,1).to(get_device())
criterion = nn.MSELoss()
acc_meter = AverageValueMeter()

optimizer = torch.optim.SGD(reg.parameters(),lr=lr, momentum=momentum)


for e in range(epochs):
    acc_meter.reset()
    reg.train()
    output = reg(X_training_norm)
    l = criterion(output.view(-1),Y_training)

    writer.add_scalar('loss/train', l.item(), global_step=e)
    
    l.backward()
    
    optimizer.step()    
    optimizer.zero_grad()
    
    reg.eval()
 
    with torch.set_grad_enabled(False):
        y_test = reg(X_testing_norm)
        l = criterion(y_test.view(-1),Y_testing)
        writer.add_scalar('loss/test', l.item(), global_step=e)